In [1]:
# !pip install wandb

In [2]:
import copy
import os
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import wandb
from PIL import Image
from sklearn.metrics import classification_report, roc_auc_score, auc, precision_recall_curve
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [3]:
CL = 'class'

TRAIN = 'train'
VAL = 'val'
TEST = 'test'

CRITERION = 'criterion'
OPTIMIZER = 'optimizer'
SCHEDULER = 'scheduler'

In [4]:
SEPARATOR = '-' * 10

In [5]:
DATA_PATH = '../input/isic-2018-task-3'

IMAGE_DIRS = {
    TRAIN: 'ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_Input',
    VAL: 'ISIC2018_Task3_Validation_Input/ISIC2018_Task3_Validation_Input',
    TEST: 'ISIC2018_Task3_Test_Input/ISIC2018_Task3_Test_Input'
}

GROND_TRUTH_PATHS = {
    TRAIN: 'ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv',
    VAL: 'ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth.csv',
    TEST: 'ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth.csv'
}

In [6]:
RANDOM_SEED = 42

In [7]:
IMAGE_MIN_SIZE = 450  # Изображения имеют размер 450 x 600
INPUT_SIZE = 380

NORMALIZATION_MEAN = [0.485, 0.456, 0.406]
NORMALIZATION_STD = [0.229, 0.224, 0.225]

CLASS_NAMES = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
NUM_CLASSES = len(CLASS_NAMES)

NUM_EPOCHS = 40
BATCH_SIZE = 16

LR = 1e-3

NUM_WORKERS = 2

In [8]:
WANDB_PROJECT_NAME = 'mediscan-efficient-net-b4'
WANDB_BASE_CONFIG = {
    'architecture': 'CNN-EfficientNet-B4',
    'dataset': 'ISIC2018-Task-3',
    'epochs': NUM_EPOCHS,
    'batch-size': BATCH_SIZE
}

---

In [9]:
class ISICDataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir  # Путь к папке с изображениями
        self.annotations = pd.read_csv(csv_file)  # Считываем csv файл, содержащий имена изображений и их метки
        self.transform = transform  # Преобразования, которые будут применены к изображениям

    def __len__(self):
        return len(self.annotations)  # Возвращаем количество изображений в датасете, согласно csv файлу с метками

    def __getitem__(self, idx):
        # Получаем полный путь к изображению: корневая папка + имя изображения + расширение
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 0] + '.jpg')

        # Открываем изображение и приводим его к RGB формату
        image = Image.open(img_path).convert('RGB')

        # Получаем метки для изображения и преобразуем их в numpy массив
        label_one_hot = self.annotations.iloc[idx, 1:].values

        # One-hot-encoded --> индекс класса
        label = np.argmax(label_one_hot)

        # Заключаем индекс класса в тензор
        label_tensor = torch.tensor(label)

        if self.transform:
            # Применяем преобразования к изображению
            image = self.transform(image)

        return image, label_tensor

---

In [10]:
# Подготовка данных
def load_data(transforms_train, transforms_eval, sampler=None):
    transform = {
        TRAIN: transforms.Compose(transforms_train),
        VAL: transforms.Compose(transforms_eval),
        TEST: transforms.Compose(transforms_eval)
    }
    
    image_datasets = {x: ISICDataset(root_dir=os.path.join(DATA_PATH, IMAGE_DIRS[x]),
                                     csv_file=os.path.join(DATA_PATH,GROND_TRUTH_PATHS[x]),
                                     transform=transform[x]) 
                      for x in [TRAIN, VAL, TEST]}
    
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS) for x in [VAL, TEST]}
    if sampler:
        dataloaders[TRAIN] = DataLoader(image_datasets[TRAIN], batch_size=BATCH_SIZE, sampler=sampler, num_workers=NUM_WORKERS)
    else:
        dataloaders[TRAIN] = DataLoader(image_datasets[TRAIN], batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    
    return image_datasets, dataloaders

In [11]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs, labels_encoded=False):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch: {epoch + 1}/{num_epochs}')

        for phase in [TRAIN, VAL]:
            if phase == TRAIN:
                model.train()  # Переводим модель в режим обучения
            else:
                model.eval()  # Переводим модель в режим оценки

            print(f'Phase: {phase}')

            epoch_loss = 0.0  # Счетчик лоссов по всем объектам за эпоху
            epoch_corrects = 0  # Счетчик верных предсказаний за эпоху

            dataloader = dataloaders[phase]
            num_iterations = len(dataloader)

            # Итерация над данными
            for i, (inputs, labels) in enumerate(dataloader):
                # Хак с возвратом каретки для Google Colab
                print(f'\rIteration: {i + 1}/{num_iterations}', end='')

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()  # Обнуляем градиенты

                # Вычисляем градиенты только если режим - TRAIN
                with torch.set_grad_enabled(phase == TRAIN):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase == TRAIN:
                        loss.backward()
                        optimizer.step()

                # Средний лосс по батчу
                batch_mean_loss = loss.item()

                # Кол-во объектов в текущем батче
                batch_size = inputs.size(0)

                # Суммарный лосс по всем объектам в батче
                batch_loss = batch_mean_loss * batch_size

                # Аккумулируем суммарные лоссы по батчам по мере продвижения по эпохе
                epoch_loss += batch_loss  # Эпоха завершится - здесь будет сумма лоссов по всем объектам всех батчей

                # Индексы максимальных значений - предсказания модели по объектам батча
                preds = torch.argmax(outputs, dim=1)
                
                if labels_encoded:
                    # Если метки закодированы в формате one-hot-encoded, получаем индексы классов. Это пригодится в эксперименте с BCEWithLogitsLoss
                    labels = torch.argmax(labels, dim=1)

                # Подсчитываем кол-во правильных предсказаний по текущему батчу
                batch_corrects = torch.sum(preds == labels)

                # Увеличиваем счетчик верных предсказаний за эпоху
                epoch_corrects += batch_corrects

                # Делим кол-во верных предсказаний на общее кол-во предсказаний в батче (сколько в батче объектов - столько и
                # было всего предсказаний) - получаем точность по текущему батчу.
                batch_acc = batch_corrects / batch_size

                if phase == TRAIN: # В режиме трейна логируем средний лосс и точность по текущему батчу в W&B
                    # Если последний батч в эпохе, то пока не коммитим (закоммитим вместе с логами по валидации)
                    commit = False if i == num_iterations - 1 else True
                    wandb.log({f'{phase}_batch_loss': batch_mean_loss, f'{phase}_batch_acc': batch_acc}, commit=commit)

            print('\rIterations completed')

            if phase == TRAIN:
                scheduler.step(batch_mean_loss) if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau) else scheduler.step()

            # Сколько объектов в датасете, столько и было всего сделано предсказаний за эпоху
            dataset_size = DATASET_SIZES[phase]

            # Делим сумму лоссов за эпоху по всем объектам датасета на кол-во объектов в датасете
            epoch_loss = epoch_loss / dataset_size  # Получаем средний лосс за эпоху

            # Делим кол-во верных предсказаний на общее кол-во сделанных предсказаний за эпоху - получаем точность
            epoch_acc = epoch_corrects.double() / dataset_size

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}', end='\n\n')

            if phase == VAL:
                # Логируем лосс и точность по валидации, а также незакоммиченные логи по последнему батчу трейна
                wandb.log({f'{phase}_loss': epoch_loss, f'{phase}_acc': epoch_acc})

                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

        print(SEPARATOR, end='\n\n')

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    
    return model, best_acc

In [12]:
def evaluate_model(model, dataloader):
    model.eval()
    total_corrects = 0

    all_labels = []
    all_preds = []
    all_probs = []

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

        total_corrects += torch.sum(preds == labels)

    accuracy = total_corrects.double() / DATASET_SIZES[TEST]
    print(f'Accuracy: {accuracy * 100:.2f}%')

    # Считаем precision, recall, f1-score
    report = classification_report(all_labels, all_preds, target_names=CLASS_NAMES)
    print(report)

    # Считаем ROC-AUC
    roc_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')
    print(f'ROC-AUC: {roc_auc:.4f}')
    
    # Считаем PR-AUC
    pr_auc_scores = []
    for class_ind in range(NUM_CLASSES):
        
        # Бинарные метки по текущему классу
        class_labels_binary = [1 if label == class_ind else 0 for label in all_labels]
        
        # Вероятности для текущего класса
        class_probs = [prob[class_ind] for prob in all_probs]
        
        precision, recall, _ = precision_recall_curve(class_labels_binary, class_probs)
        pr_auc_score = auc(recall, precision)
        pr_auc_scores.append(pr_auc_score)
    
    mean_pr_auc = np.mean(pr_auc_scores)
    print(f'PR-AUC: {mean_pr_auc:.4f}')

In [13]:
# Фиксируем сид
def set_random_seeds():
    np.random.seed(RANDOM_SEED)

    torch.manual_seed(RANDOM_SEED)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(RANDOM_SEED)

In [14]:
def init_wandb(config):
    config = {**WANDB_BASE_CONFIG, **config}
    wandb.init(project=WANDB_PROJECT_NAME, config=config)

In [15]:
def extract_params(config, component):
    component_data = config[component]
    params = {key: value for key, value in component_data.items()if key != 'class'}
    return params

In [18]:
def get_criterion(criterion_class):
    if hasattr(nn, criterion_class):
        return getattr(nn, criterion_class)
    return globals()[criterion_class]


def get_optimizer(optimizer_class):
    return getattr(optim, optimizer_class)


def get_scheduler(scheduler_class):
    return getattr(lr_scheduler, scheduler_class)

In [19]:
def run_training(config_exp, dataloaders, random_seeds=True, return_model=False, train_kwargs={}):
    if random_seeds:
        set_random_seeds()

    model = copy.deepcopy(template_model).to(device)

    init_wandb(config_exp)

    criterion = get_criterion(config_exp[CRITERION][CL])(**extract_params(config_exp, CRITERION))
    optimizer = get_optimizer(config_exp[OPTIMIZER][CL])(model.parameters(), **extract_params(config_exp, OPTIMIZER))
    scheduler = get_scheduler(config_exp[SCHEDULER][CL])(optimizer, **extract_params(config_exp, SCHEDULER))

    model, acc = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=NUM_EPOCHS, **train_kwargs)

    wandb.finish()

    return (model, acc) if return_model else acc

---

In [24]:
config = {
    CRITERION: {
        CL: 'CrossEntropyLoss'
    },
    OPTIMIZER:  {
        CL: 'Adam',
        'lr': LR
    },
    SCHEDULER: {
        CL: 'StepLR',
        'step_size': 5,
        'gamma': 1e-1,
    }
}

---

In [25]:
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [26]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [27]:
# Загружаем модель EfficientNet B4
weights = models.EfficientNet_B4_Weights.DEFAULT
template_model = models.efficientnet_b4(weights=weights)  

# Модифицируем классификатор в соответствии с актуальным количеством классов
num_features = template_model.classifier[1].in_features
template_model.classifier = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(num_features, NUM_CLASSES)  # Кол-во выходных признаков = кол-во классов
)

template_model = template_model.to(device)

---

In [28]:
# Преобразования, которые будут применены к изображениям
data_transforms_train = [transforms.Resize(INPUT_SIZE),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize(NORMALIZATION_MEAN, NORMALIZATION_STD)]

data_transforms_eval = [transforms.Resize(INPUT_SIZE),
                        transforms.ToTensor(),
                        transforms.Normalize(NORMALIZATION_MEAN, NORMALIZATION_STD)]

In [29]:
image_datasets, dataloaders = load_data(data_transforms_train, data_transforms_eval)

In [30]:
DATASET_SIZES = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

In [31]:
base_model, acc = run_training(config, dataloaders, return_model=True)

wandb: Currently logged in as: dsaperov (dsaperov-organization). Use `wandb login --relogin` to force relogin


Epoch: 1/40
Phase: train
Iterations completed
train Loss: 0.6094 Acc: 0.7848

Phase: val
Iterations completed
val Loss: 0.4349 Acc: 0.8549

----------

Epoch: 2/40
Phase: train
Iterations completed
train Loss: 0.3659 Acc: 0.8722

Phase: val
Iterations completed
val Loss: 0.3717 Acc: 0.8705

----------

Epoch: 3/40
Phase: train
Iterations completed
train Loss: 0.2636 Acc: 0.9068

Phase: val
Iterations completed
val Loss: 0.3786 Acc: 0.8756

----------

Epoch: 4/40
Phase: train
Iterations completed
train Loss: 0.2049 Acc: 0.9280

Phase: val
Iterations completed
val Loss: 0.3245 Acc: 0.8705

----------

Epoch: 5/40
Phase: train
Iterations completed
train Loss: 0.1453 Acc: 0.9486

Phase: val
Iterations completed
val Loss: 0.4185 Acc: 0.8653

----------

Epoch: 6/40
Phase: train
Iterations completed
train Loss: 0.0589 Acc: 0.9795

Phase: val
Iterations completed
val Loss: 0.3444 Acc: 0.9016

----------

Epoch: 7/40
Phase: train
Iterations completed
train Loss: 0.0365 Acc: 0.9884

Phase: val

train_batch_acc,▁▅▅▇██▇█▇███████████▇██▇████████████████
train_batch_loss,█▆▆▃▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▄▃▂▇▆▇▆█▆▆▆▅▅▅▆▄▇▇▇▆█▅▆▇▆▅▇▇▇█▆▆▆▆▅▆▇▃
val_loss,▆▃▃▁▅▂▃▆▅▇█▇▆▇█▇▆▇▆▇▅▅▅▇▆▆▅▆▆▅▆▅▆▆▅▅▇▆▇▇
train_batch_acc,1.0
train_batch_loss,0.00027
val_acc,0.87047
val_loss,0.4719


In [32]:
evaluate_model(base_model, dataloaders[TEST])

Accuracy: 86.44%
              precision    recall  f1-score   support

         MEL       0.71      0.62      0.66       171
          NV       0.91      0.96      0.93       909
         BCC       0.82      0.80      0.81        93
       AKIEC       0.56      0.70      0.62        43
         BKL       0.85      0.80      0.82       217
          DF       0.94      0.66      0.77        44
        VASC       0.87      0.74      0.80        35

    accuracy                           0.86      1512
   macro avg       0.81      0.75      0.77      1512
weighted avg       0.86      0.86      0.86      1512

ROC-AUC: 0.9733
PR-AUC: 0.8540


In [41]:
torch.save(base_model, 'base_model.pth')